# Scrapper de sites

> Récupère le contenu de chacun des URL classé par typologie (variable ndf dans le code), chargé depuis le fichier Sites.json.

> Stocke le résultat dans ndf qui conditionne la variable "fichier de sortie". Celle-ci peut être adaptée pour pointer sur une zone correcte de votre drive dans la zone "Personnalisation" dénotée ci-dessous. Les données des sites collectés sont stockées au format pickle dans le dossier /OUT/ContenusSites classé par typologie selon Sites.json.

> La variable de sortie est de type dictionnaire pour conserver les url sources (flexibilité pour pouvoir corriger). Cf. commentaires infra. 




In [ ]:
# Il faut initialiser le dossier à partir de GitHub
# cette action est à faire une seule fois lors de la première utilisation
!git clone https://github.com/Patent2net/DistorsionCom.git

Cloning into 'DistorsionCom'...
remote: Enumerating objects: 257, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 257 (delta 90), reused 217 (delta 50), pack-reused 0
Receiving objects: 100% (257/257), 41.77 MiB | 23.85 MiB/s, done.
Resolving deltas: 100% (90/90), done.


In [ ]:
stockageEntree = "/content/DistorsionCom/OUT"
stockageSortie = "/content/DistorsionCom/OUT"

In [ ]:
import os
os.chdir("DistorsionCom")
from outils import isPartner, text_from_html, myRequest


---
Personnalisation

> Ces cellules permettent de réaliser les traitements à partir de son propre espace de stockage. A n'exécuter que dans ce cas en adaptant les dossiers d'entrée et sortie. NE PAS EXECUTER SAUF A PERSONNALISER LES TRAITEMENTS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Récupérer les sorties sur son drive (créer un dossier "OUT")
stockageSortie = "/content/drive/MyDrive/"

In [ ]:
# Récupérer les entrées sur son drive
stockageEntree = "/content/drive/MyDrive/OUT"

---

In [ ]:
import requests, re, pickle, urllib
from bs4 import BeautifulSoup

In [ ]:
import json
with open(stockageEntree + '/Ressources/Sites.json', 'r', encoding='utf-8') as f:
    CatSites = json.load(f)

# Première boucle pour lever les problèmes

In [ ]:
import urllib3
urllib3.disable_warnings() # pourquoi les certificats SSL ne passent pas aujourd'hui ? 30/11 

> La dynamique du web fait que certains sites peuvent ne pas être accessibles, ne pas répondre à un instant t, ou faire planter le collecteur. Ce qui suit teste chaque URL et construit la variable BadUrl avec les urls en erreur.

changements 01/21: la fonction myRequest implémente :
1. la gestion du timeout (3 secondes), le collecteur ne devrait plus planter
2. de réssayer 3 fois en cas d'erreurs particulières (429, 502, 501)
3. ✋ ne cherche pas à vérifier le certificat SSL suite aux erreurs rencontrées ces jours ci ❗ 

In [1]:
BadUrl = dict()
Done = []
for ndc in CatSites.keys():
  if ndc not in BadUrl.keys() and ndc not in Done:
    BadUrl[ndc] = []
    print(ndc)
    for url in CatSites[ndc][0]: 
      try:
        webpage = myRequest(url)
      except:
        print("bad", url)
        BadUrl[ndc].append(url)

NameError: ignored

In [ ]:
BadUrl [ndc]

[]

In [ ]:
done = []  
# pourrait être initialisé par un os.listdir() sur le répertoire de sortie
     
for ndc in CatSites.keys():
  if ndc not in BadUrl.keys():
    BadUrl[ndc] = []
  if ndc not in done:
    soup = dict() # changement de type de données pour conserver l'URL source
  # récupère l'URL d'un site web et enregistre la page web
    for url in CatSites[ndc][0]: 
      if url not in BadUrl[ndc]:
        try:
          webpage = myRequest(url)
        except:
          BadUrl[ndc].append(url)
          continue
      else:
        continue
  # récupère le contenu de la page web à l'aide de BeautifulSoup
    #soup.append(BeautifulSoup(webpage.content, "html.parser"))
      tempoSoup = BeautifulSoup(webpage.content, "html.parser")
      if tempoSoup.title is not None:
        titre = tempoSoup.title.text 
      else:
        titre = ""
      if tempoSoup.description is not None:
        desc = tempoSoup.description.text 
      else:
        desc = ""
      texte = text_from_html(webpage.content)
      if len(titre)>0 and len(desc)>0:
        soup[url] = titre + '\n' + desc + "\n" + texte
      elif len(titre)>0:
        soup[url] = titre + "\n" + texte
      elif len(desc)>0:
        soup[url] = desc + "\n" + texte
      else:
        soup[url] = texte

      # le contenu est dans le "casier" nommé par l'url
    # nettoyage
    for cle in soup.keys():
    # soup [cle] = re.sub('^a-zA-Z0-9àâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      re.sub('^a-zA-ZàâäèéêëîïôœùûüÿçÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ', ' ', soup [cle])
      soup[cle] = soup[cle].replace("\xa0", " ")
      soup[cle] = soup[cle].replace("\n", " ")
      soup[cle] = soup[cle].replace("’", "'")
      soup[cle] = soup[cle].translate('utf8')
      dictionary = {"\\": ""}
      transtable= soup[cle].maketrans(dictionary)
      soup[cle] = soup[cle].translate(transtable)
      soup[cle] = str(soup[cle])
    
    # c'est pas tip top. Certains caractères restent. Variable selon les sites. 

    # sauvegarde sur Google Drive
    fichierDeSortie = stockageSortie + '/OUT/ContenusSites/' +CatSites [ndc][1] +'.pkl'
    with open (fichierDeSortie, 'wb') as  fictemp: #on met tous les contenus dans un pickle
      pickle.dump(soup, fictemp)
    done .append (ndc)

# Nettoyage

1. Il semble que le texte récupéré contienne de nombreux échappements d'apostrophes : `\'`
qui sont un "encodage" informatique particulier. Une substitution pour les remplacer par un apostrophe simple évitera que spacy s'embrouille.

2. BarUrl contient tous les url plantés rangés par catégorie. 

In [ ]:
import pprint as pp 

pp.pprint (BadUrl)

{'Hebergements': [],
 'Services': ['http://group-indigo.com', 'http://secretrip.pro'],
 'gouv': [],
 'locations': [],
 'mairies': [],
 'oTourism': [],
 'orga': [],
 'parcs': ['http://www.cevennes-parcnational.fr/fr',
           'http://www.mercantour-parcnational.fr/fr',
           'http://www.reunion-parcnational.fr/fr'],
 'restauration': []}


## Commentaires et exemple de ce qui est récupéré


*   les clés du dictionnaire de sortie sont les url
*   Le contenu du dictionnaire pour une clé donne le texte récupéré
* L'équivalent avec le précédent format (toutes les pages dans un texte) s'obtient facilement



In [ ]:
soup.keys()

dict_keys(['http://la-seyne.fr', 'http://le-pradet.fr', 'http://isere.fr', 'http://hyeres.fr', 'http://ville-lagarde.fr', 'http://toulon.fr', 'http://carqueiranne.fr', 'http://villedelacrau.fr'])

In [ ]:
soup ['http://la-seyne.fr']

"La Seyne.fr - Site officiel de la ville de La Seyne-sur-Mer                                                      Mairie       Vie municipale   Le maire Conseil municipal Les élus municipaux Organigramme Recueil des arrêtés municipaux       Urbanisme & Foncier   Plan local d'urbanisme AVAP Répertoire de voirie Occupation de l'Espace Public et Foncier Réglement local de publicité Formalités d'urbanisme Procès verbaux provisoires       Sécurité & Prévention   Police municipale Paiement de contravention Détention d'un chien dangereux Animal perdu/trouvé Objet perdu/trouvé Mise en fourrière d'un véhicule           Appel à projets & Marchés publics Budget & fiscalité     Economie & emploi   Recherche d'emploi Présentation économique et touristique Créer ou développer son entreprise Réglementation Améliorer son point de vente Exonérations       Annuaire Mairie         Vente publique de biens communaux      Services au public       Education Enfance & Jeunesse   Guichet unique d'inscriptions 